In [1]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load (Remember to Change These)
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()


,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [2]:
# Calculate the total number of unique schools
school_count = len(school_data_complete["school_name"].unique())
school_count

15

In [3]:
# Calculate the total number of students
student_count = school_data_complete['student_name'].count()
student_count

39170

In [4]:
# Calculate the total budget
total_budget = sum(school_data_complete['budget'].unique())
total_budget

24649428

In [5]:
# Calculate the average (mean) math score
average_math_score = school_data_complete["math_score"].mean()
average_math_score

78.98537145774827

In [6]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete["reading_score"].mean()
average_reading_score

81.87784018381414

In [7]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

74.9808526933878

In [8]:
# Calculate the percentage of students who passeed reading (hint: look at how the math percentage was calculated)  
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >= 70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

85.80546336482001

In [9]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[
    (school_data_complete["math_score"] >= 70) & (school_data_complete["reading_score"] >= 70)
].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

65.17232575950983

In [10]:

district_summary = pd.DataFrame({
    
    "Total Schools": [school_count],
    "Total Students": [student_count],
    "Total Budget": [total_budget],
    "Average Reading Score": [average_reading_score],
    "Average Math Score": [average_math_score],
    "% Passing Reading":[passing_reading_percentage],
    "% Passing Math": [passing_math_percentage],
    "Overall Passing Rate": [overall_passing_rate]

})

#store as different df to change order
dist_sum = district_summary[["Total Schools", 
                             "Total Students", 
                             "Total Budget", 
                             "Average Reading Score", 
                             "Average Math Score", 
                             '% Passing Reading', 
                             '% Passing Math', 
                             'Overall Passing Rate']]

#format cells
dist_sum.style.format({"Total Budget": "${:,.2f}", 
                       "Average Reading Score": "{:.1f}", 
                       "Average Math Score": "{:.1f}", 
                       "% Passing Math": "{:.1%}", 
                       "% Passing Reading": "{:.1%}", 
                       "Overall Passing Rate": "{:.1%}"})

,Total Schools,Total Students,Total Budget,Average Reading Score,Average Math Score,% Passing Reading,% Passing Math,Overall Passing Rate
0,15,39170,"$24,649,428.00",81.9,79.0,8580.5%,7498.1%,6517.2%


## School Summary

In [11]:
by_school = school_data_complete.set_index('school_name').groupby(['school_name'])

In [12]:
# Use the code provided to select the school type
school_types = school_data.set_index(["school_name"])["type"]

In [13]:
# Calculate the total student count
per_school_counts = by_school["Student ID"].count()

In [14]:
# Calculate the total school budget and per capita spending
per_school_budget = school_data_complete.groupby(["school_name"]).mean()["budget"]
per_school_capita = per_school_budget / per_school_counts

In [15]:
# Calculate the average test scores
per_school_math = by_school['math_score'].mean()
per_school_reading =by_school['reading_score'].mean()

In [16]:
# Calculate the number of schools with math scores of 70 or higher
school_passing_math = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count()/per_school_counts 

In [17]:
# Calculate the number of schools with reading scores of 70 or higher
school_passing_reading = school_data_complete[school_data_complete['math_score'] >= 70].groupby('school_name')['Student ID'].count()/per_school_counts 

In [18]:
# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passing_math_and_reading = school_data_complete[
    (school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)
]

In [19]:
overall = school_data_complete[(school_data_complete['reading_score'] >= 70) & (school_data_complete['math_score'] >= 70)].groupby('school_name')['Student ID'].count()/per_school_counts

In [20]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary = pd.DataFrame({
    "School Type": school_types,
    "Total Students": per_school_counts,
    "Per Student Budget": per_school_budget,
    "Total School Budget": per_school_capita,
    "Average Math Score": per_school_math,
    "Average Reading Score": per_school_reading,
    '% Passing Math': school_passing_math,
    '% Passing Reading': school_passing_reading,
    "Overall Passing Rate": overall
})




#munging
sch_summary = per_school_summary[['School Type', 
                          'Total Students', 
                          'Total School Budget', 
                          'Per Student Budget', 
                          'Average Math Score', 
                          'Average Reading Score',
                          '% Passing Math',
                          '% Passing Reading',
                          'Overall Passing Rate']]

#formatting
per_school_summary.style.format({'Total Students': '{:,}', 
                          "Total School Budget": "${:,}", 
                          "Per Student Budget": "${:.0f}",
                          'Average Math Score': "{:.1f}", 
                          'Average Reading Score': "{:.1f}", 
                          "% Passing Math": "{:.1%}", 
                          "% Passing Reading": "{:.1%}", 
                          "Overall Passing Rate": "{:.1%}"})

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Bailey High School,District,"4,976",$3124928,$628.0,77.0,81.0,66.7%,66.7%,54.6%
Cabrera High School,Charter,"1,858",$1081356,$582.0,83.1,84.0,94.1%,94.1%,91.3%
Figueroa High School,District,"2,949",$1884411,$639.0,76.7,81.2,66.0%,66.0%,53.2%
Ford High School,District,"2,739",$1763916,$644.0,77.1,80.7,68.3%,68.3%,54.3%
Griffin High School,Charter,"1,468",$917500,$625.0,83.4,83.8,93.4%,93.4%,90.6%
Hernandez High School,District,"4,635",$3022020,$652.0,77.3,80.9,66.8%,66.8%,53.5%
Holden High School,Charter,427,$248087,$581.0,83.8,83.8,92.5%,92.5%,89.2%
Huang High School,District,"2,917",$1910635,$655.0,76.6,81.2,65.7%,65.7%,53.5%
Johnson High School,District,"4,761",$3094650,$650.0,77.1,81.0,66.1%,66.1%,53.5%


## Highest-Performing Schools (by % Overall Passing)

In [21]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values("Overall Passing Rate", ascending=False)
top_schools.head(5)

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,0.941335,0.941335,0.913348
Thomas High School,Charter,1635,1043130.0,638.0,83.418349,83.848930,0.932722,0.932722,0.909480
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,0.933924,0.933924,0.905995
Wilson High School,Charter,2283,1319574.0,578.0,83.274201,83.989488,0.938677,0.938677,0.905826
Pena High School,Charter,962,585858.0,609.0,83.839917,84.044699,0.945946,0.945946,0.905405


## Bottom Performing Schools (By % Overall Passing)

In [22]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values('Overall Passing Rate')
bottom_schools.head(5)

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
school_name,,,,,,,,,
Rodriguez High School,District,3999,2547363.0,637.0,76.842711,80.744686,0.663666,0.663666,0.529882
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,0.659885,0.659885,0.532045
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,0.656839,0.656839,0.535139
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,0.667530,0.667530,0.535275
Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,0.660576,0.660576,0.535392


## Math Scores by Grade


In [23]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()

# Use the code to select only the `math_score`.
ninth_grade_math_scores = ninth_graders_scores["math_score"]
tenth_grader_math_scores = tenth_graders_scores["math_score"]
eleventh_grader_math_scores = eleventh_graders_scores["math_score"]
twelfth_grader_math_scores = twelfth_graders_scores["math_score"]

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade= pd.DataFrame({
                "9th": ninth_grade_math_scores,
               "10th": tenth_grader_math_scores,
               "11th": eleventh_grader_math_scores,
               "12th": twelfth_grader_math_scores})

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

In [24]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by "school_name" and take the mean of each.
ninth_graders_scores = ninth_graders.groupby(["school_name"]).mean()
tenth_graders_scores = tenth_graders.groupby(["school_name"]).mean()
eleventh_graders_scores = eleventh_graders.groupby(["school_name"]).mean()
twelfth_graders_scores = twelfth_graders.groupby(["school_name"]).mean()

# Use the code to select only the `reading_score`.
ninth_grade_reading_scores = ninth_graders_scores["reading_score"]
tenth_grader_reading_scores = tenth_graders_scores["reading_score"]
eleventh_grader_reading_scores = eleventh_graders_scores["reading_score"]
twelfth_grader_reading_scores = twelfth_graders_scores["reading_score"]

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({
                "9th": ninth_grade_math_scores,
               "10th": tenth_grader_math_scores,
               "11th": eleventh_grader_math_scores,
               "12th": twelfth_grader_math_scores})

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Scores by School Spending

In [25]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [26]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()

In [27]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(per_school_capita, spending_bins, labels=labels)
school_spending_df

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,Spending Ranges (Per Student)
school_name,,,,,,,,,,
Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,0.666801,0.666801,0.546423,$585-630
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,0.941335,0.941335,0.913348,<$585
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,0.659885,0.659885,0.532045,$630-645
Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,0.683096,0.683096,0.542899,$630-645
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,0.933924,0.933924,0.905995,$585-630
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,0.667530,0.667530,0.535275,$645-680
Holden High School,Charter,427,248087.0,581.0,83.803279,83.814988,0.925059,0.925059,0.892272,<$585
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,0.656839,0.656839,0.535139,$645-680
Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,0.660576,0.660576,0.535392,$645-680


In [28]:
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Overall Passing Rate"]

In [29]:
# Assemble into DataFrame 
spending_summary = pd.DataFrame({
          "Average Math Score" : spending_math_scores,
          "Average Reading Score": spending_reading_scores,
          "% Passing Math": spending_passing_math,
          "% Passing Reading": spending_passing_reading,
          "% Overall Passing": overall_passing_spending})
# Display results
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,0.934601,0.934601,0.903695
$585-630,81.899826,83.155286,0.871335,0.871335,0.814186
$630-645,78.518855,81.624473,0.734842,0.734842,0.628577
$645-680,76.997210,81.027843,0.661648,0.661648,0.535269


## Scores by School Size

In [30]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [31]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.

per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], size_bins, labels=labels)
per_school_summary

,School Type,Total Students,Per Student Budget,Total School Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate,School Size
school_name,,,,,,,,,,
Bailey High School,District,4976,3124928.0,628.0,77.048432,81.033963,0.666801,0.666801,0.546423,Large (2000-5000)
Cabrera High School,Charter,1858,1081356.0,582.0,83.061895,83.975780,0.941335,0.941335,0.913348,Medium (1000-2000)
Figueroa High School,District,2949,1884411.0,639.0,76.711767,81.158020,0.659885,0.659885,0.532045,Large (2000-5000)
Ford High School,District,2739,1763916.0,644.0,77.102592,80.746258,0.683096,0.683096,0.542899,Large (2000-5000)
Griffin High School,Charter,1468,917500.0,625.0,83.351499,83.816757,0.933924,0.933924,0.905995,Medium (1000-2000)
Hernandez High School,District,4635,3022020.0,652.0,77.289752,80.934412,0.667530,0.667530,0.535275,Large (2000-5000)
Holden High School,Charter,427,248087.0,581.0,83.803279,83.814988,0.925059,0.925059,0.892272,Small (<1000)
Huang High School,District,2917,1910635.0,655.0,76.629414,81.182722,0.656839,0.656839,0.535139,Large (2000-5000)
Johnson High School,District,4761,3094650.0,650.0,77.072464,80.966394,0.660576,0.660576,0.535392,Large (2000-5000)


In [32]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["Overall Passing Rate"]

In [33]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary` 
size_summary = pd.DataFrame({
          "Average Math Score" : size_math_scores,
          "Average Reading Score": size_reading_scores,
          "% Passing Math": size_passing_math,
          "% Passing Reading": size_passing_reading,
          "% Overall Passing": size_overall_passing})
# Display results
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,0.935502,0.935502,0.898839
Medium (1000-2000),83.374684,83.864438,0.935997,0.935997,0.906215
Large (2000-5000),77.746417,81.344493,0.699634,0.699634,0.582860


## Scores by School Type

In [60]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = per_school_summary.groupby(["School Type"]).mean()["Average Math Score"]
type_reading_scores = per_school_summary.groupby(["School Type"]).mean()["Average Reading Score"]
type_passing_math = per_school_summary.groupby(["School Type"]).mean()["% Passing Math"]
type_passing_reading = per_school_summary.groupby(["School Type"]).mean()["% Passing Reading"]
type_overall_passing = per_school_summary.groupby(["School Type"]).mean()["Overall Passing Rate"]

In [61]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.DataFrame({
    "Average Math Score": type_math_scores,
    "Average Reading Score": type_reading_scores,
    "% Passing Math": type_passing_math,
    "% Passing Reading": type_passing_reading,
    "% Overall Passing": type_overall_passing
})


# Display results
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,0.936208,0.936208,0.904322
District,76.956733,80.966636,0.665485,0.665485,0.536722
